In [1]:
import torch
import torch.onnx

# Загрузка предобученной модели YOLOv5 из PyTorch
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Запуск модели
input_tensor = torch.rand(1, 3, 640, 640)  # Пример входных данных
output = model(input_tensor)

/usr/local/lib/python3.10/dist-packages/torch/hub.py:293: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


requirements: Ultralytics requirements ['gitpython>=3.1.30', 'pillow>=10.3.0', 'requests>=2.32.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 194.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 212.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0

requirements: AutoUpdate success ✅ 9.2s, installed 3 packages: ['gitpython>=3.1.30', 'pillow>=10.3.0', 'requests>=2.32.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-5-28 Python-3.10.12 torch-2.3.0+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 29.9MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [2]:
print(output)

tensor([[[4.85606e+00, 5.07196e+00, 8.93911e+00,  ..., 1.06075e-03, 7.76958e-04, 3.94752e-03],
         [1.35028e+01, 6.01625e+00, 2.57713e+01,  ..., 1.41154e-03, 9.19218e-04, 4.02650e-03],
         [1.81560e+01, 5.25513e+00, 3.01260e+01,  ..., 1.39446e-03, 9.49694e-04, 4.94210e-03],
         ...,
         [5.62432e+02, 5.98728e+02, 1.47796e+02,  ..., 8.14691e-03, 1.22773e-03, 1.81089e-03],
         [5.86527e+02, 6.03949e+02, 1.12569e+02,  ..., 9.06425e-03, 1.60292e-03, 2.09188e-03],
         [6.16292e+02, 6.17362e+02, 1.14256e+02,  ..., 7.99317e-03, 1.62799e-03, 2.09675e-03]]])


In [4]:
!pip install onnx onnxruntime

# Конвертация модели в ONNX формат
torch.onnx.export(model, input_tensor, "yolov5.onnx", verbose=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 13.2 MB/s eta 0:00:00


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:666: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  y = self.model(im, augment=augment, visualize=visualize) if augment or visualize else self.model(im)
/root/.cache/torch/hub/ultralytics_yolov5_master/models/yolo.py:100: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.dynamic or self.grid[i].shape[2:4] != x[i].shape[2:4]:


In [6]:
import onnx
import onnxruntime

# Загрузка модели из файла ONNX
onnx_model = onnx.load("yolov5.onnx")
ort_session = onnxruntime.InferenceSession("yolov5.onnx")

In [10]:
# Запуск модели из файла ONNX
ort_inputs = {ort_session.get_inputs()[0].name: input_tensor.numpy()}
ort_outs = ort_session.run(None, ort_inputs)
print("Результат запуска модели RNN в ONNX:", ort_outs)

Результат запуска модели RNN в ONNX: [array([[[     4.8561,       5.072,      8.9391, ...,   0.0010608,  0.00077698,   0.0039475],
        [     13.503,      6.0163,      25.771, ...,   0.0014116,  0.00091922,   0.0040266],
        [     18.156,      5.2551,      30.126, ...,   0.0013944,  0.00094968,   0.0049421],
        ...,
        [     562.43,      598.73,       147.8, ...,   0.0081469,   0.0012277,   0.0018109],
        [     586.53,      603.95,      112.57, ...,   0.0090643,   0.0016029,   0.0020919],
        [     616.29,      617.36,      114.26, ...,   0.0079932,    0.001628,   0.0020967]]], dtype=float32)]


In [11]:
!pip install onnx2torch
from onnx2torch.converter import convert
torch_model = convert("yolov5.onnx")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.1/80.1 kB 2.7 MB/s eta 0:00:00


In [13]:
output = torch_model(input_tensor)
print(output)

tensor([[[4.85606e+00, 5.07196e+00, 8.93911e+00,  ..., 1.06075e-03, 7.76958e-04, 3.94752e-03],
         [1.35028e+01, 6.01626e+00, 2.57713e+01,  ..., 1.41154e-03, 9.19218e-04, 4.02650e-03],
         [1.81560e+01, 5.25513e+00, 3.01260e+01,  ..., 1.39446e-03, 9.49693e-04, 4.94210e-03],
         ...,
         [5.62432e+02, 5.98728e+02, 1.47796e+02,  ..., 8.14692e-03, 1.22773e-03, 1.81089e-03],
         [5.86527e+02, 6.03949e+02, 1.12569e+02,  ..., 9.06424e-03, 1.60292e-03, 2.09187e-03],
         [6.16292e+02, 6.17362e+02, 1.14256e+02,  ..., 7.99317e-03, 1.62799e-03, 2.09674e-03]]], grad_fn=<CatBackward0>)
